In [ ]:
# ==============================
# BLOCK 1 — Imports & User Config
# ==============================
import pandas as pd
import numpy as np
import re
from pathlib import Path
import statsmodels.formula.api as smf
import statsmodels.api as sm
import pickle
import warnings
warnings.filterwarnings("ignore")

# ===== PATHS =====
STEP4_DIR = Path(r" ")
STEP5_DIR = Path(r" ")
STEP5_DIR.mkdir(parents=True, exist_ok=True)

# ===== CATEGORY DICT (Option 1: Each person → Exactly one category) =====
CATEGORY_DICT = {
    # Politicians & world leaders
    "Politicians and World Leaders": [
        "Abdelfattah Elsisi","Alberto Fernández","Barack Obama","Benjamin Netanyahu","Boris Johnson",
        "Emmanuel Macron","Imran Khan","Ivan Duque","Jair Bolsonaro","Joko Widodo","Justin Trudeau",
        "Kaguta Museveni","King Salman","Lopez Obrador","Mohammed AlMaktoum","Moon Jaein",
        "Muhammadu Buhari","Nana AkufoAddo","Narendra Modi","Nicolas Maduro","Paul Kagame","Queen Rania","Sebastian Pinera","Tayyip Erdogan"
    ],

    # Business / entrepreneurs
    "Business Leaders and Entrepreneurs": [
        "Bill Gates","Elon Musk","Eric Yuan","Jack Dorsey","Joe Gebbia","John Collison","Ken Fisher",
        "Marc Benioff","Melinda Gates","Michael Dell","Micky Arison","Mike Bloomberg","Mike Brookes",
        "Orlando Bravo","Patrick Collison","Ralph Lauren","Ricardo Salinas","Samuel Bankman",
        "Tilman Fertitta","Tim Sweeney","Tobi Lutke","Vinod Khosla"
    ],

    # Entertainment umbrella: actors, athletes, comedians, models
    "Entertainers and Celebrities": [
        # singers, actors, athletes, comedians, reality stars
        "Adele Adkins","Alecia Beth","Alicia Keys","Armando Perez","Britney Spears","Bruno Mars",
        "Chris Brown","Demi Lovato","Drake Graham","Harry Styles","Jennifer Lopez","Justin Bieber",
        "Justin Timberlake","Kanye West","Katy Perry","Lady Gaga","Liam Payne","Lil Wayne",
        "Louis Tomlinson","Miley Cyrus","Niall Horan","Nicki Minaj","Robyn Rihanna","Shakira Ripoll",
        "Shawn Mendes","Taylor Swift","Wiz Khalifa","Zayn Malik",
        "Akshay Kumar","Amitabh Bachchan","Deepika Padukone","Emma Watson","Hrithik Roshan",
        "Priyanka Chopra","Salman Khan","Selena Gomez","ShahRukh Khan",
        "Andres Iniesta","Cristiano Ronaldo","LeBron James","Mesut Ozil","Neymar Junior",
        "Ricardo Kaka","Sachin Tendulkar","Virat Kohli",
        "Conan O Brien","Jimmy Fallon","Kevin Hart","Oprah Winfrey","Patrick Harris","Whindersson Nunes",
        "Kendal Jenner","Khloe Kardashian","Kim Kardashian","Kourtney Kardashian","Kylie Jenner"
    ]
}

FOLLOWERS_DICT = {
    "Abdelfattah Elsisi": 6300000,
    "Alberto Fernández": 2200000,
    "Barack Obama": 129000000,
    "Benjamin Netanyahu": 3500000,
    "Boris Johnson": 4500000,
    "Emmanuel Macron": 10200000,
    "Imran Khan": 21200000,
    "Ivan Duque": 2600000,
    "Jair Bolsonaro": 14100000,
    "Joko Widodo": 21700000,
    "Justin Trudeau": 6600000,
    "Kaguta Museveni": 3600000,
    "King Salman": 10200000,
    "Lopez Obrador": 11000000,
    "Mohammed AlMaktoum": 10900000,
    "Moon Jaein": 2100000,
    "Muhammadu Buhari": 4500000,
    "Nana AkufoAddo": 2700000,
    "Narendra Modi": 108600000,
    "Nicolas Maduro": 4700000,
    "Paul Kagame": 3300000,
    "Queen Rania": 9700000,
    "Sebastian Pinera": 2300000,
    "Tayyip Erdogan": 21300000,
    "Adele Adkins": 25500000,
    "Alecia Beth": 28700000,
    "Alicia Keys": 27700000,
    "Armando Perez": 22800000,
    "Britney Spears": 51600000,
    "Bruno Mars": 40900000,
    "Chris Brown": 30500000,
    "Demi Lovato": 50000000,
    "Drake Graham": 37800000,
    "Harry Styles": 35300000,
    "Jennifer Lopez": 42200000,
    "Justin Bieber": 105700000,
    "Justin Timberlake": 57400000,
    "Kanye West": 32700000,
    "Katy Perry": 101300000,
    "Lady Gaga": 79700000,
    "Liam Payne": 31600000,
    "Lil Wayne": 32900000,
    "Louis Tomlinson": 33500000,
    "Miley Cyrus": 44600000,
    "Niall Horan": 38200000,
    "Nicki Minaj": 27800000,
    "Robyn Rihanna": 105800000,
    "Shakira Ripoll": 51300000,
    "Shawn Mendes": 25200000,
    "Taylor Swift": 91400000,
    "Wiz Khalifa": 34700000,
    "Zayn Malik": 29200000,
    "Akshay Kumar": 46600000,
    "Amitabh Bachchan": 48300000,
    "Deepika Padukone": 25700000,
    "Emma Watson": 25700000,
    "Hrithik Roshan": 31800000,
    "Priyanka Chopra": 26800000,
    "Salman Khan": 45200000,
    "Selena Gomez": 63600000,
    "ShahRukh Khan": 43500000,
    "Andres Iniesta": 24300000,
    "Cristiano Ronaldo": 114200000,
    "LeBron James": 52000000,
    "Mesut Ozil": 25300000,
    "Neymar Junior": 62900000,
    "Ricardo Kaka": 27400000,
    "Sachin Tendulkar": 40500000,
    "Virat Kohli": 67500000,
    "Conan O Brien": 26100000,
    "Jimmy Fallon": 47800000,
    "Kevin Hart": 35200000,
    "Oprah Winfrey": 39400000,
    "Patrick Harris": 23100000,
    "Whindersson Nunes": 27600000,
    "Kendal Jenner": 30600000,
    "Khloe Kardashian": 29200000,
    "Kim Kardashian": 73300000,
    "Kourtney Kardashian": 25200000,
    "Kylie Jenner": 38900000,
    "Bill Gates": 66000000,
    "Elon Musk": 228500000,
    "Eric Yuan": 86800,
    "Jack Dorsey": 6400000,
    "Joe Gebbia": 206100,
    "John Collison": 208500,
    "Ken Fisher": 428300,
    "Marc Benioff": 1100000,
    "Melinda Gates": 2400000,
    "Michael Dell": 796100,
    "Micky Arison": 184800,
    "Mike Bloomberg": 2500000,
    "Mike Brookes": 105100,
    "Orlando Bravo": 44100,
    "Patrick Collison": 572000,
    "Ralph Lauren": 2200000,
    "Ricardo Salinas": 2100000,
    "Samuel Bankman": 996800,
    "Tilman Fertitta": 103800,
    "Tim Sweeney": 316100,
    "Tobi Lutke": 424100,
    "Vinod Khosla": 686500

    # ... include all others or read from a CSV later
}

AGE_DICT = {
    "Abdelfattah Elsisi": 66,
    "Alberto Fernández": 62,
    "Barack Obama": 60,
    "Benjamin Netanyahu": 72,
    "Boris Johnson": 57,
    "Emmanuel Macron": 43,
    "Imran Khan": 69,
    "Ivan Duque": 45,
    "Jair Bolsonaro": 66,
    "Joko Widodo": 60,
    "Justin Trudeau": 49,
    "Kaguta Museveni": 77,
    "King Salman": 85,
    "Lopez Obrador": 68,
    "Mohammed AlMaktoum": 72,
    "Moon Jaein": 68,
    "Muhammadu Buhari": 79,
    "Nana AkufoAddo": 77,
    "Narendra Modi": 71,
    "Nicolas Maduro": 58,
    "Paul Kagame": 64,
    "Queen Rania": 51,
    "Sebastian Pinera": 71,
    "Tayyip Erdogan": 67,
    "Adele Adkins": 33,
    "Alecia Beth": 42,
    "Alicia Keys": 40,
    "Armando Perez": 40,
    "Britney Spears": 39,
    "Bruno Mars": 36,
    "Chris Brown": 32,
    "Demi Lovato": 29,
    "Drake Graham": 35,
    "Harry Styles": 27,
    "Jennifer Lopez": 52,
    "Justin Bieber": 27,
    "Justin Timberlake": 40,
    "Kanye West": 44,
    "Katy Perry": 37,
    "Lady Gaga": 35,
    "Liam Payne": 28,
    "Lil Wayne": 39,
    "Louis Tomlinson": 29,
    "Miley Cyrus": 28,
    "Niall Horan": 28,
    "Nicki Minaj": 38,
    "Robyn Rihanna": 33,
    "Shakira Ripoll": 44,
    "Shawn Mendes": 23,
    "Taylor Swift": 31,
    "Wiz Khalifa": 34,
    "Zayn Malik": 28,
    "Akshay Kumar": 54,
    "Amitabh Bachchan": 79,
    "Deepika Padukone": 35,
    "Emma Watson": 31,
    "Hrithik Roshan": 47,
    "Priyanka Chopra": 39,
    "Salman Khan": 55,
    "Selena Gomez": 29,
    "ShahRukh Khan": 56,
    "Andres Iniesta": 37,
    "Cristiano Ronaldo": 36,
    "LeBron James": 36,
    "Mesut Ozil": 34,
    "Neymar Junior": 29,
    "Ricardo Kaka": 39,
    "Sachin Tendulkar": 48,
    "Virat Kohli": 33,
    "Conan O Brien": 58,
    "Jimmy Fallon": 47,
    "Kevin Hart": 42,
    "Oprah Winfrey": 67,
    "Patrick Harris": 48,
    "Whindersson Nunes": 26,
    "Kendal Jenner": 26,
    "Khloe Kardashian": 37,
    "Kim Kardashian": 41,
    "Kourtney Kardashian": 42,
    "Kylie Jenner": 24,
    "Bill Gates": 66,
    "Elon Musk": 50,
    "Eric Yuan": 51,
    "Jack Dorsey": 44,
    "Joe Gebbia": 40,
    "John Collison": 31,
    "Ken Fisher": 70,
    "Marc Benioff": 57,
    "Melinda Gates": 57,
    "Michael Dell": 56,
    "Micky Arison": 72,
    "Mike Bloomberg": 79,
    "Mike Brookes": 41,
    "Orlando Bravo": 51,
    "Patrick Collison": 33,
    "Ralph Lauren": 82,
    "Ricardo Salinas": 66,
    "Samuel Bankman": 29,
    "Tilman Fertitta": 64,
    "Tim Sweeney": 51,
    "Tobi Lutke": 41,
    "Vinod Khosla": 66

    # ... include all others or read from a CSV later
}

JOIN_DICT = {
    "Abdelfattah Elsisi": 2014,
    "Alberto Fernández": 2010,
    "Barack Obama": 2007,
    "Benjamin Netanyahu": 2008,
    "Boris Johnson": 2015,
    "Emmanuel Macron": 2013,
    "Imran Khan": 2010,
    "Ivan Duque": 2009,
    "Jair Bolsonaro": 2010,
    "Joko Widodo": 2011,
    "Justin Trudeau": 2008,
    "Kaguta Museveni": 2010,
    "King Salman": 2013,
    "Lopez Obrador": 2009,
    "Mohammed AlMaktoum": 2009,
    "Moon Jaein": 2011,
    "Muhammadu Buhari": 2014,
    "Nana AkufoAddo": 2011,
    "Narendra Modi": 2009,
    "Nicolas Maduro": 2013,
    "Paul Kagame": 2009,
    "Queen Rania": 2009,
    "Sebastian Pinera": 2008,
    "Tayyip Erdogan": 2009,
    "Adele Adkins": 2010,
    "Alecia Beth": 2009,
    "Alicia Keys": 2009,
    "Armando Perez": 2009,
    "Britney Spears": 2008,
    "Bruno Mars": 2009,
    "Chris Brown": 2010,
    "Demi Lovato": 2009,
    "Drake Graham": 2009,
    "Harry Styles": 2010,
    "Jennifer Lopez": 2009,
    "Justin Bieber": 2009,
    "Justin Timberlake": 2009,
    "Kanye West": 2010,
    "Katy Perry": 2009,
    "Lady Gaga": 2008,
    "Liam Payne": 2010,
    "Lil Wayne": 2010,
    "Louis Tomlinson": 2009,
    "Miley Cyrus": 2011,
    "Niall Horan": 2010,
    "Nicki Minaj": 2009,
    "Robyn Rihanna": 2009,
    "Shakira Ripoll": 2009,
    "Shawn Mendes": 2011,
    "Taylor Swift": 2008,
    "Wiz Khalifa": 2009,
    "Zayn Malik": 2010,
    "Akshay Kumar": 2009,
    "Amitabh Bachchan": 2010,
    "Deepika Padukone": 2010,
    "Emma Watson": 2010,
    "Hrithik Roshan": 2010,
    "Priyanka Chopra": 2009,
    "Salman Khan": 2010,
    "Selena Gomez": 2009,
    "ShahRukh Khan": 2010,
    "Andres Iniesta": 2009,
    "Cristiano Ronaldo": 2010,
    "LeBron James": 2009,
    "Mesut Ozil": 2012,
    "Neymar Junior": 2010,
    "Ricardo Kaka": 2009,
    "Sachin Tendulkar": 2010,
    "Virat Kohli": 2009,
    "Conan O Brien": 2010,
    "Jimmy Fallon": 2008,
    "Kevin Hart": 2009,
    "Oprah Winfrey": 2009,
    "Patrick Harris": 2009,
    "Whindersson Nunes": 2010,
    "Kendal Jenner": 2010,
    "Khloe Kardashian": 2009,
    "Kim Kardashian": 2009,
    "Kourtney Kardashian": 2009,
    "Kylie Jenner": 2011,
    "Bill Gates": 2009,
    "Elon Musk": 2009,
    "Eric Yuan": 2012,
    "Jack Dorsey": 2006,
    "Joe Gebbia": 2007,
    "John Collison": 2007,
    "Ken Fisher": 2017,
    "Marc Benioff": 2009,
    "Melinda Gates": 2010,
    "Michael Dell": 2009,
    "Micky Arison": 2011,
    "Mike Bloomberg": 2008,
    "Mike Brookes": 2008,
    "Orlando Bravo": 2013,
    "Patrick Collison": 2007,
    "Ralph Lauren": 2009,
    "Ricardo Salinas": 2009,
    "Samuel Bankman": 2019,
    "Tilman Fertitta": 2013,
    "Tim Sweeney": 2013,
    "Tobi Lutke": 2007,
    "Vinod Khosla": 2009

    # ... include all others or read from a CSV later
}

YEARS_ACTIVE_DICT = {p: 2021 - year for p, year in JOIN_DICT.items()}

def normalize_person_key(name: str) -> str:
    if not isinstance(name, str):
        return ""
    
    # ============ HARD FIXES FOR THE TWO PROBLEMATIC NAMES ============
    hard_fixes = {
        "ConanOBrien":       "Conan O'Brien",     # correct with apostrophe
        "AbdelfattahElsisi": "Abdelfattah Elsisi", # ← FIXED: capital E and S !!
    }
    if name in hard_fixes:
        return hard_fixes[name]
    
    # ============ STANDARD CAMELCASE → SPACE FOR EVERYONE ELSE ============
    name = re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', name)   # insert space before capitals
    name = re.sub(r'\s+', ' ', name).strip()           # collapse spaces
    
    # ============ EXTRA SAFETY (in case regex ever breaks again) ============
    if name == "Conan O Brien":
        return "Conan O'Brien"
    if name == "Abdelfattah El sisi":
        return "Abdelfattah Elsisi"
    
    return name


# Build person → category mapping (Option 1)
category_norm = {}
for cat, people in CATEGORY_DICT.items():
    for p in people:
        category_norm[normalize_person_key(p)] = cat

print("BLOCK 1 loaded.")

In [ ]:
# ==============================
# BLOCK 2 — Build combined_long
# ==============================

def read_elite_summary(file_path: Path, person_name: str):
    try:
        df = pd.read_excel(file_path, engine="openpyxl")
    except Exception as e:
        print(f"ERROR reading {file_path}: {e}")
        return None

    df.columns = [str(c).strip() for c in df.columns]
    metric_col = df.columns[0]
    df = df.rename(columns={metric_col: "metric"})

    for col in ["Full", "COVID", "COVID/Full", "p_two_sided", "obs_value",
                "null_mean", "null_std", "n_perm", "ratio_to_null_mean"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    df["person"] = person_name
    return df

rows = []
files_found = 0

for person_dir in STEP4_DIR.iterdir():
    if not person_dir.is_dir():
        continue

    person_name = person_dir.name
    candidate = None

    for f in person_dir.iterdir():
        if f.is_file() and person_name in f.name and "Centrality" in f.name:
            candidate = f
            break

    if candidate is None:
        xs = list(person_dir.glob("*.xlsx"))
        if xs:
            candidate = xs[0]

    if candidate is None:
        print(f"⚠️ No file found in {person_dir}")
        continue

    df_e = read_elite_summary(candidate, person_name)
    if df_e is not None:
        rows.append(df_e)
        files_found += 1

combined_long = pd.concat(rows, ignore_index=True)

# normalize column names
rename_map = {}
for c in combined_long.columns:
    lc = c.lower()
    if lc == "full": rename_map[c] = "full"
    if lc == "covid": rename_map[c] = "covid"
    if lc in ["covid/full", "covid_full_ratio"]: rename_map[c] = "covid_full_ratio"
    if lc == "p_two_sided": rename_map[c] = "p_two_sided"

combined_long = combined_long.rename(columns=rename_map)
combined_long["metric"] = combined_long["metric"].str.strip().str.lower()

combined_long.to_csv(STEP5_DIR/"combined_person_metric_long.csv", index=False, encoding="utf-8-sig")
print("combined_long saved.")

In [ ]:
# ========================================
# BLOCK 3 — Merge full metrics + COVID + metadata
# ========================================

# 1. Full-period metrics in wide format
full_wide = (
    combined_long[["person", "metric", "full"]]
    .dropna(subset=["full"])
    .pivot_table(index="person", columns="metric", values="full", aggfunc="first")
)
full_wide.columns = [f"full_{c}" for c in full_wide.columns]
full_wide = full_wide.reset_index()

# 2. COVID-period rows (only rows that actually have a COVID value)
covid_rows = combined_long[
    combined_long["covid"].notna()
].copy()

covid_rows = covid_rows[[
    "person", "metric", "full", "covid", "covid_full_ratio",
    "p_two_sided", "obs_value", "null_mean", "null_std"
]]

# 3. Merge COVID rows with full-period metrics
merged = covid_rows.merge(full_wide, on="person", how="left")

# 4. Normalise person names ONCE
merged["person_norm"] = merged["person"].apply(normalize_person_key)

# 5. Normalised metadata dictionaries
followers_norm = {normalize_person_key(k): v for k, v in FOLLOWERS_DICT.items()}
age_norm       = {normalize_person_key(k): v for k, v in AGE_DICT.items()}
years_norm     = {normalize_person_key(k): v for k, v in YEARS_ACTIVE_DICT.items()}

# 6. Build metadata frame (only for people that appear in the data)
meta = pd.DataFrame({"person_norm": merged["person_norm"].unique()})
meta["followers"]     = meta["person_norm"].map(followers_norm)
meta["age"]           = meta["person_norm"].map(age_norm)
meta["years_active"]  = meta["person_norm"].map(years_norm)
meta["category"]      = meta["person_norm"].map(category_norm)

# 7. Merge metadata in
merged = merged.merge(meta, on="person_norm", how="left")

# 8. Save
merged.to_csv(
    STEP5_DIR / "merged_person_metric_covidsummary.csv",
    index=False,
    encoding="utf-8-sig"
)
print(f"Block 3 complete → {len(merged):,} rows in merged table")
print(f"   Unique elites: {merged['person'].nunique()}")
print(f"   Categories present: {merged['category'].nunique()}")

In [ ]:
# ==============================
# BLOCK 4 — Coverage summary
# ==============================

metrics = sorted(merged["metric"].unique())
summary_rows = []

for m in metrics:
    sub = merged[merged["metric"]==m]
    n = sub.shape[0]
    sig = sub["p_two_sided"].lt(0.05).sum()
    marginal = sub["p_two_sided"].lt(0.10).sum()
    mean_ratio = sub["covid_full_ratio"].replace([np.inf,-np.inf], np.nan).mean()

    summary_rows.append({
        "metric": m,
        "n_elites_with_covid_ego": n,
        "n_significant_p<0.05": sig,
        "n_marginal_p<0.1": marginal,
        "mean_ratio": mean_ratio
    })

summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv(STEP5_DIR/"metric_coverage_summary.csv", index=False)
summary_df

In [5]:
# ==============================
# BLOCK 5 — OLS regressions
# ==============================
REG_DIR = STEP5_DIR / "regression_results"
REG_DIR.mkdir(exist_ok=True)

reg_results = []

MIN_SIG = 10  # minimum significant elites

for m in metrics:
    sub = merged[merged["metric"]==m].copy()
    sub = sub[sub["covid_full_ratio"].notna()]

    sig = sub[sub["p_two_sided"] < 0.05]
    if len(sig) < MIN_SIG:
        reg_results.append({
            "metric": m, "run": False,
            "reason": "Too few significant elites"
        })
        continue

    df = sig.copy()
    df["log_followers"] = np.log1p(df["followers"].astype(float))

    full_cols = [c for c in df.columns if c.startswith("full_")]
    core_full = ["full_density","full_clustering","full_modularity",
                 "full_centralization_top3","full_n_nodes","full_n_edges",
                 "full_avg_degree","full_avg_closeness","full_avg_pagerank"]
    used_full = [c for c in core_full if c in df.columns]

    rhs = ["log_followers","age","years_active","C(category)"] + used_full
    rhs = [c for c in rhs if c in df.columns]

    formula = "covid_full_ratio ~ " + " + ".join(rhs)
    df2 = df[["covid_full_ratio"]+rhs].dropna()
    if df2.shape[0] < len(rhs)+3:
        reg_results.append({
            "metric": m, "run": False, "reason": "Insufficient rows after NA drop"
        })
        continue

    model = smf.ols(formula, df2).fit(cov_type="HC3")

    with open(REG_DIR/f"{m}_reg_summary.txt","w") as f:
        f.write(model.summary().as_text())

    reg_results.append({
        "metric": m, "run": True,
        "r2_adj": model.rsquared_adj,
        "n_used": df2.shape[0],
        "formula": formula
    })

pd.DataFrame(reg_results).to_csv(STEP5_DIR/"regression_overview.csv", index=False)

In [ ]:
# ==============================
# BLOCK 5 — OLS regressions (WITH TEXT FILES SAVED)
# ==============================
REG_DIR = STEP5_DIR / "regression_results"
REG_DIR.mkdir(exist_ok=True)

reg_results = []
MIN_SIG = 10  # minimum significant elites

for m in metrics:
    sub = merged[merged["metric"] == m].copy()
    sub = sub[sub["covid_full_ratio"].notna()]
    sig = sub[sub["p_two_sided"] < 0.05]
    
    if len(sig) < MIN_SIG:
        reg_results.append({"metric": m, "run": False, "reason": "Too few significant elites"})
        continue

    df = sig.copy()
    df["log_followers"] = np.log1p(df["followers"].astype(float))

    full_cols = [c for c in df.columns if c.startswith("full_")]
    core_full = ["full_density","full_clustering","full_modularity",
                 "full_centralization_top3","full_n_nodes","full_n_edges",
                 "full_avg_degree","full_avg_closeness","full_avg_pagerank"]
    used_full = [c for c in core_full if c in df.columns]
    rhs = ["log_followers","age","years_active","C(category)"] + used_full
    rhs = [c for c in rhs if c in df.columns]
    formula = "covid_full_ratio ~ " + " + ".join(rhs)

    df2 = df[["covid_full_ratio"] + rhs].dropna()
    if df2.shape[0] < len(rhs) + 3:
        reg_results.append({"metric": m, "run": False, "reason": "Insufficient rows after NA drop"})
        continue

    model = smf.ols(formula, df2).fit(cov_type="HC3")

    # SAVE THE FULL REGRESSION SUMMARY TEXT FILE (this was missing!)
    txt_path = REG_DIR / f"{m}_reg_summary.txt"
    with open(txt_path, "w", encoding="utf-8") as f:
        f.write(model.summary().as_text())

    # Also save a nice HTML version (optional but pretty)
    html_path = REG_DIR / f"{m}_reg_summary.html"
    with open(html_path, "w", encoding="utf-8") as f:
        f.write(model.summary().as_html())

    reg_results.append({
        "metric": m,
        "run": True,
        "r2_adj": model.rsquared_adj,
        "n_used": df2.shape[0],
        "formula": formula,
        "p_values": {param: model.pvalues[param] for param in model.params.index}
    })

pd.DataFrame(reg_results).to_csv(STEP5_DIR / "regression_overview.csv", index=False)
print(f"Block 5 finished! → {len([r for r in reg_results if r['run']])} regressions saved in:\n   {REG_DIR}")

In [ ]:
# ==============================
# BLOCK 6 — Category-level analysis (robust version)
# ==============================
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import os

cat_results = []
tukey_dir = STEP5_DIR / "Tukey_posthoc"
tukey_dir.mkdir(exist_ok=True)

print("Starting category-level analysis...")

for m in metrics:
    # 1. Select data for this metric
    sub = merged[
        (merged["metric"] == m) & 
        (merged["covid_full_ratio"].notna())
    ].copy()

    # 2. CRITICAL: Drop rows where category is missing
    if sub["category"].isna().any():
        n_dropped = sub["category"].isna().sum()
        print(f"   → {m}: Dropping {n_dropped} rows with missing category")
        sub = sub.dropna(subset=["category"])

    # Need at least 2 categories AND at least 2 observations per category
    if sub["category"].nunique() < 2:
        continue

    # Quick check: each category should have ≥2 observations (Tukey requires this)
    counts = sub["category"].value_counts()
    if (counts < 2).any():
        valid_cats = counts[counts >= 2].index
        sub = sub[sub["category"].isin(valid_cats)]
        if sub["category"].nunique() < 2:
            continue

    # ————— ANOVA —————
    model = ols("covid_full_ratio ~ C(category)", data=sub).fit()
    anova_table = anova_lm(model, typ=2)

    anova_p = float(anova_table["PR(>F)"].iloc[0])
    cat_results.append({
        "metric": m,
        "n_elites": len(sub),
        "n_categories": sub["category"].nunique(),
        "anova_p": anova_p,
        "anova_sig": "Yes" if anova_p < 0.05 else "No"
    })

    # ————— Tukey HSD (only if ANOVA suggests differences) —————
    if anova_p < 0.10:  # you can relax to 0.10 or keep 0.05
        try:
            tukey = pairwise_tukeyhsd(
                endog=sub["covid_full_ratio"],
                groups=sub["category"],
                alpha=0.05
            )
            tukey_path = tukey_dir / f"Tukey_{m.replace('/', '_')}.txt"
            with open(tukey_path, "w", encoding="utf-8") as f:
                f.write(tukey.summary().as_text())
            print(f"   → Tukey saved for {m}")
        except Exception as e:
            print(f"   → Tukey failed for {m}: {e}")
    else:
        print(f"   → {m}: ANOVA not suggestive (p={anova_p:.3f}), skipping Tukey")

# Save summary table
results_df = pd.DataFrame(cat_results)
results_df = results_df.sort_values("anova_p")
results_df.to_csv(STEP5_DIR / "category_anova_results.csv", index=False)

print("\nBlock 6 finished!")
print(f"   ANOVA results saved → {len(results_df)} metrics tested")
print(f"   Significant at p<0.05 → {results_df['anova_p'].lt(0.05).sum()}")
results_df.head(10)

In [8]:
# ==============================
# BLOCK 7 — Mediation analysis
# ==============================
from statsmodels.stats.mediation import Mediation

# pick the metric: avg_closeness
sub = merged[merged["metric"]=="avg_closeness"].copy()
sub = sub[sub["p_two_sided"]<0.05].copy()
sub = sub.dropna(subset=["covid_full_ratio","followers","full_density"])

sub["log_followers"] = np.log1p(sub["followers"])

# mediator model M ~ X
mediator_model = smf.ols("full_density ~ log_followers + age + C(category)", data=sub)

# outcome model Y ~ X + M
outcome_model = smf.ols("covid_full_ratio ~ log_followers + full_density + age + C(category)", data=sub)

med = Mediation(outcome_model, mediator_model, "log_followers", "full_density").fit()

with open(STEP5_DIR/"mediation_followers_density_closeness.txt","w") as f:
    f.write(str(med.summary()))

In [ ]:
# ==============================
# BLOCK 8 — Master Excel output
# ==============================
writer = pd.ExcelWriter(STEP5_DIR/"MASTER_REPORT.xlsx", engine="openpyxl")

combined_long.to_excel(writer, sheet_name="combined_long", index=False)
merged.to_excel(writer, sheet_name="merged", index=False)
summary_df.to_excel(writer, sheet_name="coverage_summary", index=False)
pd.DataFrame(reg_results).to_excel(writer, sheet_name="regression_overview", index=False)
pd.DataFrame(cat_results).to_excel(writer, sheet_name="category_ANOVA", index=False)

writer.close()

print("MASTER_REPORT.xlsx created.")

In [ ]:
# ================================================================
# BLOCK 9 — Corrected Category-Difference Plot + Full Significance Heatmap
# ================================================================
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

PLOT_DIR = STEP5_DIR / "visualizations"
PLOT_DIR.mkdir(exist_ok=True)

# ------------------------------------------------------------
# Helper: signed-log transform
# ------------------------------------------------------------
def signed_log(x, eps=1e-9):
    x = np.asarray(x, float)
    return np.sign(x) * np.log10(np.abs(x) + eps)

# ------------------------------------------------------------
# A. CATEGORY-DIFFERENCE FIGURE (all 9 metrics, fixed widths + dashed grid)
# ------------------------------------------------------------
anova_res = pd.read_csv(STEP5_DIR/"category_anova_results.csv")
all_metrics = anova_res["metric"].tolist()   # use ALL 9 metrics

plot_df = merged[merged["metric"].isin(all_metrics)].copy()
plot_df["signedlog_ratio"] = signed_log(plot_df["covid_full_ratio"])

categories = [
    "Politicians and World Leaders",
    "Entertainers and Celebrities",
    "Business Leaders and Entrepreneurs",
]
palette = sns.color_palette("Set2", 3)

plt.figure(figsize=(20, 8))

# Draw violinplot
sns.violinplot(
    data=plot_df,
    x="metric",
    y="signedlog_ratio",
    hue="category",
    split=False,
    inner=None,
    palette=palette,
    cut=0
)

# Draw *narrower* grouped boxplots manually
x_positions = np.arange(len(all_metrics))
offsets = [-0.25, 0, 0.25]
box_width = 0.05

for i, cat in enumerate(categories):
    subset = plot_df[plot_df["category"] == cat]
    sns.boxplot(
        data=subset,
        x="metric",
        y="signedlog_ratio",
        positions=x_positions + offsets[i],
        width=box_width,
        showcaps=True,
        boxprops={"facecolor": "white", "zorder": 3},
        showfliers=False,
        whiskerprops={"linewidth": 1},
        medianprops={"color": "black", "linewidth": 1.5}
    )

# ----------------------
# ★ Add light dashed grid
# ----------------------
plt.grid(
    axis='y',
    linestyle='--',
    linewidth=0.6,
    alpha=0.35
)

plt.title("Category Differences in COVID-Era Discourse Restructuring", fontsize=17)
plt.xlabel("Metric", fontsize=14)
plt.ylabel("Signed log₁₀(COVID/Full Ratio)", fontsize=14)
plt.xticks(rotation=45, ha="right")
plt.legend(title="Category", bbox_to_anchor=(1.02, 1), loc="upper left")
plt.tight_layout()
plt.savefig(PLOT_DIR/"Fig_Category_Differences_FIXED2.png", dpi=450)
plt.show()


# =============================================================
# B. FULL REGRESSION SIGNIFICANCE HEATMAP (all predictors included)
# =============================================================
reg_overview = pd.read_csv(STEP5_DIR/"regression_overview.csv")

# All dependent variables (all 9)
dependent_vars = [
    "avg_closeness","avg_degree","avg_pagerank","centralization_top3",
    "clustering","density","modularity","n_edges","n_nodes"
]

# All independent predictors you listed
predictors = [
    "log_followers","age","years_active",
    "full_density","full_clustering","full_modularity",
    "full_centralization_top3","full_n_nodes","full_n_edges",
    "full_avg_degree","full_avg_closeness","full_avg_pagerank"
]

heat_rows = []

for dep in dependent_vars:
    summary_file = REG_DIR / f"{dep}_reg_summary.txt"
    if not summary_file.exists():
        continue
    with open(summary_file, "r") as f:
        lines = f.readlines()

    collect = False
    for line in lines:
        if "coef" in line and "std err" in line:
            collect = True
            continue
        if collect:
            if line.strip() == "":
                break
            parts = line.split()
            if len(parts) < 5:
                continue

            var = parts[0]
            if var not in predictors:
                continue

            pval = float(parts[-1])
            if pval < 0.001:
                sig = 3
            elif pval < 0.01:
                sig = 2
            elif pval < 0.05:
                sig = 1
            else:
                sig = 0

            heat_rows.append([dep, var, sig])

heat_df = pd.DataFrame(heat_rows, columns=["dependent", "predictor", "sig"])
heat_map = heat_df.pivot(index="dependent", columns="predictor", values="sig").fillna(0)

plt.figure(figsize=(18, 10))
sns.heatmap(
    heat_map,
    cmap="YlGnBu",
    linewidths=0.6,
    linecolor="white",
    annot=True,
    fmt=".0f",
    cbar_kws={
        "label": "Significance Level\n(3: p < 0.001, 2: p < 0.01, 1: p < 0.05, 0: ns ≥ 0.05)"
    }
)

plt.title("Regression Significance Across Predictors", fontsize=18)
plt.xlabel("Independent Predictor", fontsize=14)
plt.ylabel("Dependent Variable", fontsize=14)
plt.tight_layout()
plt.savefig(PLOT_DIR/"Fig_Regression_Significance_Heatmap_FULL.png", dpi=450)
plt.show()

print("✓ Updated figures saved.")